# Word2vec preprocessing

Preprocessing is not the most exciting part of NLP, but it is still one of the most important ones. Your task is to preprocess raw text (you can use your own, or [this one](http://mattmahoney.net/dc/text8.zip). For this task text preprocessing mostly consists of:

1. cleaning (mostly, if your dataset is from social media or parsed from the internet)
1. tokenization
1. building the vocabulary and choosing its size. Use only high-frequency words, change all other words to UNK or handle it in your own manner. You can use `collections.Counter` for that.
1. assigning each token a number (numericalization). In other words, make word2index и index2word objects.
1. data structuring and batching - make X and y matrices generator for word2vec (explained in more details below)

**ATTN!:** If you use your own data, please, attach a download link. 

Your goal is to make SkipGramBatcher class which returns two numpy tensors with word indices. It should be possible to use one for word2vec training. You can implement batcher for Skip-Gram or CBOW architecture, the picture below can be helpful to remember the difference.

![text](https://raw.githubusercontent.com/deepmipt/deep-nlp-seminars/651804899d05b96fc72b9474404fab330365ca09/seminar_02/pics/architecture.png)

There are several ways to do it right. Shapes could be `x_batch.shape = (batch_size, 2*window_size)`, `y_batch.shape = (batch_size,)` for CBOW or `(batch_size,)`, `(batch_size,)` for Skip-Gram. You should **not** do negative sampling here.

They should be adequately parametrized: CBOW(window_size, ...), SkipGram(window_size, ...). You should implement only one batcher in this task; and it's up to you which one to chose.

Useful links:
1. [Word2Vec Tutorial - The Skip-Gram Model](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/)
1. [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
1. [Distributed Representations of Words and Phrases and their Compositionality](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

You can write the code in this notebook, or in a separate file. It can be reused for the next task. The result of your work should represent that your batch has a proper structure (right shapes) and content (words should be from one context, not some random indices). To show that, translate indices back to words and print them to show something like this:

```
text = ['first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including']

window_size = 2

# CBOW:
indices_to_words(x_batch) = \
        [['first', 'used', 'early', 'working'],
        ['used', 'against', 'working', 'class'],
        ['against', 'early', 'class', 'radicals'],
        ['early', 'working', 'radicals', 'including']]

indices_to_words(labels_batch) = ['against', 'early', 'working', 'class']

# Skip-Gram

indices_to_words(x_batch) = ['against', 'early', 'working', 'class']

indices_to_words(labels_batch) = ['used', 'working', 'early', 'radicals']]

```

If you struggle with something, ask your neighbor. If it is not obvious for you, probably someone else is looking for the answer too. And in contrast, if you see that you can help someone - do it! Good luck!

In [1]:
from collections import Counter
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from operator import itemgetter
import numpy as np

In [2]:
with open('text8','r',encoding='utf8') as f:
    data = f.read()
print(data[:20])

 anarchism originate


In [3]:
words = word_tokenize(data)
print(words[:20])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english']


In [4]:
counted = Counter(words)

In [5]:
print(counted.most_common(10))

[('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430), ('two', 192644)]


In [6]:
print(sorted(counted.items(), key=itemgetter(1), reverse=False)[180000:180050])

[('brau', 4), ('buryats', 4), ('eshira', 4), ('gruzim', 4), ('italkim', 4), ('koyukon', 4), ('mentawai', 4), ('minangkabau', 4), ('myene', 4), ('parsi', 4), ('rappahannock', 4), ('shangaan', 4), ('yakut', 4), ('lfur', 4), ('jealousies', 4), ('steim', 4), ('shamela', 4), ('brust', 4), ('ewds', 4), ('photocopied', 4), ('waukegan', 4), ('clenching', 4), ('empathogen', 4), ('oma', 4), ('fotw', 4), ('redoubtable', 4), ('dateformat', 4), ('spiele', 4), ('lumbricus', 4), ('lumbricidae', 4), ('megascolecidae', 4), ('waterlogged', 4), ('basswood', 4), ('dusts', 4), ('sovereignties', 4), ('canandaigua', 4), ('chemung', 4), ('chenango', 4), ('xylose', 4), ('slowhand', 4), ('surbiton', 4), ('bluesbreakers', 4), ('engrish', 4), ('nmci', 4), ('bethnal', 4), ('missives', 4), ('quam', 4), ('oenoanda', 4), ('philodemus', 4), ('kohima', 4)]


In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
vocab = [word[0] for word in counted.items() if word[0] not in stop_words and word[1]>1]

In [9]:
print(len(vocab))
print(vocab[:50])

135179
['anarchism', 'originated', 'term', 'abuse', 'first', 'used', 'early', 'working', 'class', 'radicals', 'including', 'diggers', 'english', 'revolution', 'sans', 'culottes', 'french', 'whilst', 'still', 'pejorative', 'way', 'describe', 'act', 'violent', 'means', 'destroy', 'organization', 'society', 'also', 'taken', 'positive', 'label', 'self', 'defined', 'anarchists', 'word', 'derived', 'greek', 'without', 'archons', 'ruler', 'chief', 'king', 'political', 'philosophy', 'belief', 'rulers', 'unnecessary', 'abolished', 'although']


In [10]:
word2index = { word : i for i, word in enumerate(vocab)}
index2word = { i : word for i, word in enumerate(vocab)}

In [11]:
print(word2index['anarchism'])

0


In [12]:
print(index2word[0])

anarchism


In [13]:
class CBOWBatcher():
    def __init__(self,vocab,batch_size=4,window_size=2):
        self.batch_size=batch_size
        self.window_size=window_size
        self.word2index = { word : i for i, word in enumerate(vocab)}
        self.index2word = { i : word for i, word in enumerate(vocab)} 
        
    def make_batches(self,text):
        x_batches, y_batches = [],[]
        x_batch, y_batch = [], []
        for i, word in enumerate(text):
            if i<self.window_size:
                words_left = ['START' for i in range(self.window_size-i)]+text[:i]
            else:
                words_left = text[i-self.window_size:i]
            words_right = text[i+1:i+self.window_size+1]
            if len(words_right) < self.window_size:
                words_right = words_right + ['END' for i in range(self.window_size-len(words_right))]
            x_indexed = [self.word2index.get(word_in_window,-1) for word_in_window in words_left+words_right]
            if len(x_batch)==self.batch_size:
                x_batches.append(np.array(x_batch))
                y_batches.append(np.array(y_batch))
                x_batch, y_batch = [], []
            x_batch.append(x_indexed)
            y_batch.append(self.word2index.get(word,-1))   
        x_batches.append(np.array(x_batch))
        y_batches.append(np.array(y_batch))
        return x_batches, y_batches
    
    def indices_to_words(self,batch):
        batch_ndim = batch.ndim
        if batch_ndim == 1: # y_batch
            return [self.index2word.get(y,'UNK') for y in batch]
        else:
            if batch_ndim == 2: # x_batch
                out_batch = []
                for window in batch:
                    out_window = []
                    for x in window:
                        out_window.append(self.index2word.get(x,'UNK'))
                    out_batch.append(out_window)
                return out_batch
            else:
                raise ValueError("Wrong dimension")

In [14]:
batcher = CBOWBatcher(vocab)
tokenized_text = ['first', 'used', 'against', 'early',
                  'working', 'class', 'radicals', 'including'] 
x_batches, y_batches = batcher.make_batches(tokenized_text)

for x_batch, y_batch in zip(x_batches, y_batches):
    print(batcher.indices_to_words(x_batch))
    print(batcher.indices_to_words(y_batch))


[['UNK', 'UNK', 'used', 'UNK'], ['UNK', 'first', 'UNK', 'early'], ['first', 'used', 'early', 'working'], ['used', 'UNK', 'working', 'class']]
['first', 'used', 'UNK', 'early']
[['UNK', 'early', 'class', 'radicals'], ['early', 'working', 'radicals', 'including'], ['working', 'class', 'including', 'UNK'], ['class', 'radicals', 'UNK', 'UNK']]
['working', 'class', 'radicals', 'including']
